In [1]:
from transformers import __version__ as tversion
print(tversion)

4.5.1


In [2]:
! pip install -U jax jaxlib dm_haiku optax
#!pip install --upgrade pip
# Installs the wheel compatible with CUDA 11 and cuDNN 8.2 or newer.
# ! pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_releases.html  # Note: wheels only available on linux.
## !pip install --upgrade jax jaxlib==0.3.0+cuda110 -f https://storage.googleapis.com/jax-releases/jax_releases.html
#! pip install dm_haiku

     |████████████████████████████████| 912 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 65.4 MB 49.9 MB/s eta 0:00:01
     |████████████████████████████████| 309 kB 51.5 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 49.0 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 2.8 MB/s  eta 0:00:01
  Created wheel for jax: filename=jax-0.3.1-py3-none-any.whl size=1054277 sha256=b24f52ddd45657863ea2a32e00fbae2a2136552697483d1b80c93d74780a5552
  Stored in directory: /root/.cache/pip/wheels/04/14/e8/ee9de500f173ec900a5167686d9bb17c0171ed678680b96a57
Successfully built jax
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.1.70
    Uninstalling jaxlib-0.1.70:
      Successfully uninstalled jaxlib-0.1.70
  Attempting uninstall:

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_github = user_secrets.get_secret("github")
! rm -rf feedback2021
! git clone https://{secret_github}@github.com/VilmosProkaj/feedback2021.git


Cloning into 'feedback2021'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 200 (delta 98), reused 153 (delta 51), pack-reused 0
Receiving objects: 100% (200/200), 56.58 KiB | 2.36 MiB/s, done.
Resolving deltas: 100% (98/98), done.


In [ ]:
! ls feedback2021/src/feedback2021 -R

In [4]:
! pip install feedback2021/
#! rm -rf feedback2021/.git

Processing ./feedback2021
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for feedback2021: filename=feedback2021-0.0.post1.dev27+gaff12e9-py3-none-any.whl size=22678 sha256=616ece5cddb232dba817a78e57e0fd03166b4586b0cd8587b8c061af771eacb8
  Stored in directory: /tmp/pip-ephem-wheel-cache-kjs9p5qg/wheels/58/ad/18/0ce3edd06dac18258cb4965c488a42e46fa829de9c6aa2319e
Successfully built feedback2021


In [5]:
# %env XLA_PYTHON_CLIENT_MEM_FRACTION=.75
compute_on_tpu = True
just_test = False
if compute_on_tpu:
    if just_test:
        import os
        os.environ['XLA_FLAGS'] = '--xla_force_host_platform_device_count=2'
    else:
        from feedback2021.jax_tpu_init import jax_tpu_init
        jax_tpu_init()
import jax
jax.devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [6]:
# jax_tpu_init??

In [ ]:
LOG_TO_WANDB = False

In [ ]:
if LOG_TO_WANDB:
    !pip install --upgrade wandb -q # experiment tracking

In [ ]:
if LOG_TO_WANDB:
    import wandb
    import os
    os.environ["WANDB_PROJECT"] = "kaggle_feedback"
    os.environ["WANDB_ENTITY"] = "prvi"
    os.environ["WANDB_LOG_MODEL"] = "true"
    os.environ["WANDB_WATCH"] = "gradient"

    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        api_key = user_secrets.get_secret("wandb")
        os.environ["WANDB_API_KEY"] = api_key
        wandb.login()
        wandb.init(dir="/tmp/") 
    except:
        print('If you want to use your W&B account, '
              'go to Add-ons -> Secrets and provide your W&B access token.\n'
              'Use the Label name `wandb`. \n'
              'Get your W&B access token from here: https://wandb.ai/authorize')

In [6]:
from feedback2021.helper import id2label, label2id

from feedback2021.prepare_data import (create_train_dataset_pd, 
                                       to_chunk_data, 
                                       chunk_mapping,
                                       add_input_ids, 
                                       add_labels,
                                       add_rle_word2,
                                       has_name,
                                    )

import feedback2021.metric as metric

from feedback2021.postprocess import (mk_metric, 
                                      mk_prediction_transform, 
                                      mk_binary_metric, 
                                      mk_binary_prediction_transform)

from feedback2021.visualize import show_result

import feedback2021.hk_roberta as hk_roberta

In [7]:
# CONFIG
#just_test = False
experiment_id = 1
task = "token_classification"
model_checkpoint = "distilroberta-base" 
# "allenai/longformer-base-4096" 
# "distilroberta-base" # "microsoft/deberta-v3-xsmall" #"roberta-base"
max_length = 512
stride = 128
min_tokens = 6
model_path = f'{model_checkpoint.split("/")[-1]}-{experiment_id}'
data_from_wandb = False
save_to_wandb = False or not data_from_wandb


## Data Preprocessing

In [8]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
#import tensorflow.config
#tensorflow.config.experimental.set_visible_devices([], "GPU")

In [9]:
from feedback2021.clean_train_data import mk_clean_train_data
cleaned_train = mk_clean_train_data()
if just_test:
    cleaned_train = cleaned_train[:100]
data = create_train_dataset_pd(cleaned_train_df=cleaned_train, 
                        tokenizer=tokenizer, 
                        verbose=True)

from sklearn.model_selection import train_test_split
data = dict(zip(['train','test'], train_test_split(data, 
                                                   test_size=0.1, 
                                                   shuffle=True, 
                                                   random_state=42)))
# data = data.train_test_split(test_size=0.1, shuffle=True, seed=42)
data

2022-02-23 05:18:15.905093: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-02-23 05:18:20.666281: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-02-23 05:18:20.666338: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)


reformat train data:   0%|          | 0/144293 [00:00<?, ?it/s]

reading essays:   0%|          | 0/15594 [00:00<?, ?it/s]

searching for disaligned labels:   0%|          | 0/15594 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (959 > 512). Running this sequence through the model will result in indexing errors


    word mapping:   0%|          | 0/15594 [00:00<?, ?it/s]

adding token rle:   0%|          | 0/15594 [00:00<?, ?it/s]

 adding label:   0%|          | 0/15594 [00:00<?, ?it/s]

{'train':                                                            text  \
 id                                                                
 B4FD96336393  In this story the author is trying to claim th...   
 49BF71773C4D  Dear Senator,\n\nWe should be changing to the ...   
 D485BE90E01E  It just seems like yeasterday that technogoly ...   
 131DA85211F7  When looking for important advice you usually ...   
 E8301922006D  The human emotion is one of the most importnt ...   
 ...                                                         ...   
 199980ADEC34  Dear TEACHER_NAME,\n\nI believe that Policy 1 ...   
 B6261FF12B56  By reading this article I am in favor of keepi...   
 1BEB13D46E9D  I think the United States should keep the Elec...   
 2BD0B513E03D  Distance Learning\n\nSome schools offer distan...   
 CB648246281B  February 15,2011\n\nTo Whom It May Concern:\n\...   
 
                                                        rle_char  \
 id                                  

In [ ]:
if LOG_TO_WANDB and save_to_wandb:
    data.remove_columns([#'input_ids', 
                         #'rle_token', 
                         'labels', 
                         #'offset_mapping'
    ]).save_to_disk('data')

    artifact = wandb.Artifact('data', description='train test split', type='dataset')
    artifact.add_dir('data')
    wandb.log_artifact(artifact)
    !ls -sRh data
    

In [ ]:
# if  not isinstance(data.column_names, dict):
#     data = data.train_test_split(test_size=0.1, shuffle=True, seed=42)
    
# chunk_data = to_chunk_data(data, 
#                            chunk_len=max_length, 
#                            stride=stride, 
#                            prefix=[tokenizer.bos_token_id],
#                            postfix=[tokenizer.eos_token_id])
# chunk_data

In [10]:
def chop_up(data, tokenizer, max_length, stride):
    examples = {k: list(data[k]) for k in data.columns}
    examples['labels'] = list(data['labels'])
    examples['id'] = list(data.index.values)
    f=chunk_mapping(chunk_len=max_length, 
              stride=stride, 
              prefix=[tokenizer.bos_token_id],
              postfix=[tokenizer.eos_token_id])
    return f(examples)

def to_records(data):
    return [dict(zip(data.keys(), rec))  for rec in zip(*data.values())]
    

In [11]:
chunk_data = {
    k: chop_up(v, tokenizer, max_length=max_length, stride=stride)
    for k,v in data.items()
}

In [12]:
import yaml
print(yaml.dump(
    {
        k: {
            k0: f'size={len(v0[0])}, type={type(v0[0][0]).__name__}' 
            if isinstance(v0[0], list) else v0[0] 
            for k0, v0 in v.items()
        } 
     for k,v in chunk_data.items()
    }
))

test:
  input_ids: size=512, type=int
  labels: size=512, type=int
  offset: 0
  text_id: D110D1D93270
train:
  input_ids: size=249, type=int
  labels: size=249, type=int
  offset: 0
  text_id: B4FD96336393



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# if not has_name(data, 'rle_word'):
#     from feedback2021.helper import Block
#     def add_rle_word2(data):
#         # assert has_name(data, 'offset_mapping'), 'add input_ids first!'
#         if not has_name(data,'word_mapping'):
#             data = add_word_mapping(data)
#         return data.map(lambda x: {'rle_word': [Block(t).inv_map(x['word_mapping']) 
#                                                  for t in x['rle_char']]},
#                             desc='rle to word coordinates')
#     data = add_rle_word2(data)

In [ ]:
# from collections import defaultdict
# import numpy as np
# word_counts = defaultdict(list)
# from  datasets import concatenate_datasets
# all_data = concatenate_datasets(list(data.values()))
# for x in all_data['rle_word']:
#     for cls_id, start, end  in x:
#         word_counts[id2label[cls_id]].append(end-start)

# for k, v in word_counts.items():
#     plt.hist(v,bins=np.arange(1,max(v)+1)-0.5)
#     plt.title(f'{k} min:{min(v)} max: {max(v)}, 2%,5%: {np.percentile(v,[2,5])}')
#     plt.show()

## Model and Training

In [ ]:
# chunk_data = chunk_data.rename_column('labels','label')
# DataLoader?

In [ ]:
# import numpy as np
# def mk_collate_fn(tokenizer, max_length=512):
#     def collate_fn(features):
        
#         batch ={key: np.array([f[key]+[t]*(max_length-len(f[key])) for f in features], dtype=np.int32)
#                 for t,key in zip([tokenizer.pad_token_id,-100],['input_ids','labels'])}
#         return batch
#     return collate_fn
# from torch.utils.data import DataLoader

# train_dataset = DataLoader(chunk_data_list, shuffle=True, #.remove_columns(['offset', 'text_id']), 
#                            batch_size=16,
#                            collate_fn=mk_collate_fn(tokenizer, 512))

In [13]:
%%time
import tensorflow.data as tfdata
#ds=
import numpy as np

def mk_collate_fn(tokenizer, max_length=512):
    def collate_fn(features):
        
        batch = features.copy()
        for t, key in zip([tokenizer.pad_token_id,-100], ['input_ids','labels']):
            batch[key] = np.array([f+[t]*(max_length-len(f)) for f in batch[key]], 
                                  dtype=np.int32)
        return batch
    
    return collate_fn

    
collate_fn = mk_collate_fn(tokenizer, max_length)

ds ={k: tfdata.Dataset.from_tensor_slices(collate_fn(v)) for k,v in chunk_data.items()}
#(chunk_data_list)

CPU times: user 24.9 s, sys: 6.81 s, total: 31.7 s
Wall time: 17 s


In [14]:
batch_size = 32
dummy_data = tfdata.Dataset.from_tensor_slices(
    {
        'input_ids': np.zeros((batch_size, max_length), dtype=np.int32),
        'labels': -100*np.ones((batch_size, max_length), dtype=np.int32),
        'text_id': np.array(['0'*11]*batch_size),
        'offset': np.array(np.zeros(batch_size, dtype=np.int32))
    }
)

In [15]:
dummy_data

<TensorSliceDataset shapes: {input_ids: (512,), labels: (512,), text_id: (), offset: ()}, types: {input_ids: tf.int32, labels: tf.int32, text_id: tf.string, offset: tf.int32}>

In [16]:
def subset(x):
    return {k: x[k] for k in ['input_ids', 'labels', 'offset']}

train_dataset = (ds['train'].
                 map(subset).
                 repeat().
                 shuffle(1000).
                 batch(batch_size=batch_size).
                 batch(batch_size=jax.device_count()).as_numpy_iterator())

test_dataset = (ds['test'].
                concatenate(dummy_data.repeat(jax.device_count())).
                map(subset).batch(batch_size=batch_size, drop_remainder=True).
                batch(batch_size=jax.device_count()).as_numpy_iterator())

In [17]:
len(ds['test']), len(ds['train'])

(4806, 43955)

In [21]:
class Metric:
    def __init__(self):
        self.reset()
    def reset(self):
        self._value = 0
        self._n = 0
    def update(self,v):
        self._n += 1
        self._value += v
    def value(self):
        return self._value/self._n

In [18]:
{k: f'shape={v.shape}, dtype={v.dtype}' for k,v in next(iter(train_dataset)).items()}

{'input_ids': 'shape=(8, 32, 512), dtype=int32',
 'labels': 'shape=(8, 32, 512), dtype=int32',
 'offset': 'shape=(8, 32), dtype=int32'}

In [19]:
local_device_count = jax.device_count()
local_device_count

8

In [20]:
import haiku as hk
import jax.numpy as jnp
import optax
import functools
from typing import Any, Mapping

class Updater:
  """A stateless abstraction around an init_fn/update_fn pair.
  This extracts some common boilerplate from the training loop.
  """

  def __init__(self, net_init, loss_fn,
               optimizer: optax.GradientTransformation):
    self._net_init = net_init
    self._loss_fn = loss_fn
    self._opt = optimizer

  @functools.partial(jax.jit, static_argnums=0)
  def init(self, rng, data, pretrained_params=None):
    """Initializes state of the updater."""
    out_rng, init_rng = jax.random.split(rng)
    params = self._net_init(init_rng, data)
    if pretrained_params is not None:
        params = hk.data_structures.merge(params, pretrained_params)
    #params = hk.data_structures.map(lambda x: jnp.stack([x]*local_device_count), params)
    opt_state = self._opt.init(params)
    # rng = jax.random.PRNGKey(FLAGS.train_init_random_seed)
    #rng = jnp.broadcast_to(rng, (local_device_count,) + rng.shape)
    out = dict(
        step=np.array(0),
        rng=out_rng,
        opt_state=opt_state,
        params=params,
        loss=np.array(0),
    )
    return out

  @functools.partial(jax.jit, static_argnums=0)
  def update(self, state: Mapping[str, Any], data: Mapping[str, jnp.ndarray]):
    """Updates the state using some data and returns metrics."""
    rng, new_rng = jax.random.split(state['rng'])
    params = state['params']
    loss, grads = jax.value_and_grad(self._loss_fn)(params, rng, data)
    
    if compute_on_tpu:
        grads = jax.lax.pmean(grads, 'i')
    
    
    updates, opt_state = self._opt.update(grads, state['opt_state'])
    params = optax.apply_updates(params, updates)

    new_state = {
        'step': state['step'] + 1,
        'rng': new_rng,
        'opt_state': opt_state,
        'params': params,
        'loss': state['loss'] + loss
    }

#     metrics = {
#         'step': state['step'],
#         'loss': loss,
#     }
    return new_state #, metrics
    
#     def reset_loss(self, state):
#         state['loss'] = 0*state['loss']
#         return state

In [21]:
import logging
import time
from transformers import AutoConfig
from feedback2021.hk_roberta import (build_forward_fn, 
                                     lm_loss_fn, 
                                     # GradientUpdater, 
                                     hk, optax, functools, Mapping, jnp, jax)

## global variables
# batch_size = 16  # Train batch size per core
learning_rate = 2.5e-5 # Max learning-rate
grad_clip_value = 0.15  # Gradient norm clip value

checkpoint_dir = '/jax-transformer'  # Directory to store checkpoints
LOG_EVERY = 250
MAX_STEPS = 9000
n_epoch = 3

config = AutoConfig.from_pretrained('distilroberta-base', 
                                    label2id=label2id, 
                                    id2label=id2label).to_dict()
translation = hk_roberta.weight_name_translation(config=config, prefix='roberta')
pretrained_params = hk_roberta.load_hf_pytorch_weights('distilroberta-base', translation)
logging.info = print
logging.info('Starting...')

forward_fn = build_forward_fn(config)

forward_fn = hk.transform(forward_fn)

loss_fn = functools.partial(lm_loss_fn, forward_fn.apply, config)

lr_schedule=optax.warmup_cosine_decay_schedule(init_value=0, 
                                               peak_value=1, 
                                               warmup_steps=500, 
                                               decay_steps=8500, 
                                               end_value=1e-4)

optimizer = optax.chain(
        optax.clip_by_global_norm(grad_clip_value),
        optax.adam(learning_rate, b1=0.9, b2=0.99),
        optax.scale_by_schedule(lr_schedule),
    )

updater = Updater(forward_fn.init, loss_fn, optimizer)

# Initialize parameters.
logging.info('Initializing parameters...')
rng = jax.random.PRNGKey(428)
data = next(iter(train_dataset))
state = updater.init(rng, #jnp.broadcast_to(rng, (local_device_count,) + rng.shape), 
                     {k:v[0] for k,v in data.items()},
                     pretrained_params=pretrained_params)
if compute_on_tpu:
    state = jax.device_put_replicated(state, jax.devices())
    update = jax.pmap(updater.update, axis_name='i')
else:
    update = updater.update
    
logging.info('Starting train loop...')
prev_time=start_time= time.time()
# loss = Metric()
train_iter = iter(train_dataset)
#for epoch in range(n_epoch):
for step  in range(MAX_STEPS): #enumerate(train_dataset):
        data = next(train_iter)
        state = update(state, data)
        # loss.update(float(metrics['loss']))
        # We use JAX runahead to mask data preprocessing and JAX dispatch overheads.
        # Using values from state/metrics too often will block the runahead and can
        # cause these overheads to become more prominent.
        if step % LOG_EVERY == 0:
            tdelta = (time.time() - prev_time)
            telapsed = (time.time() - start_time)
            steps_per_sec = LOG_EVERY / tdelta
            log={
                'step': int(state['step'][0]), 
                'loss': float(state['loss'].mean()), 
                'steps_per_sec': steps_per_sec, 
                'elapsed_time': telapsed
            }
            logging.info(log)
            state['loss'] = 0*state['loss']
            prev_time = time.time()

Downloading:   0%|          | 0.00/331M [00:00<?, ?B/s]

unused torch weights: roberta.embeddings.token_type_embeddings.weight,
roberta.pooler.dense.weight, roberta.pooler.dense.bias, lm_head.bias,
lm_head.dense.weight, lm_head.dense.bias, lm_head.layer_norm.weight,
lm_head.layer_norm.bias, lm_head.decoder.weight
Starting...
Initializing parameters...
Starting train loop...
{'step': 1, 'loss': 2.0178143978118896, 'steps_per_sec': 5.967741518735171, 'elapsed_time': 41.89189887046814}
{'step': 251, 'loss': 308.63153076171875, 'steps_per_sec': 2.1578482934216994, 'elapsed_time': 157.82719492912292}
{'step': 501, 'loss': 152.59725952148438, 'steps_per_sec': 3.4508323330868307, 'elapsed_time': 230.29325246810913}
{'step': 751, 'loss': 131.8741455078125, 'steps_per_sec': 3.414926899188497, 'elapsed_time': 303.52902460098267}
{'step': 1001, 'loss': 116.84872436523438, 'steps_per_sec': 3.442193594278414, 'elapsed_time': 376.1770763397217}
{'step': 1251, 'loss': 106.08416748046875, 'steps_per_sec': 3.4520746796776844, 'elapsed_time': 448.613075494766

In [23]:
params = hk.data_structures.map(lambda name,module,x: x[0], state['params'])
import pickle
with open('params.pkl','wb') as f:
    pickle.dump(params, f)

{'embedding/layernorm': {'offset': DeviceArray([-1.44100815e-01,  2.59286109e-02, -9.05191805e-03,
                5.17673325e-03,  1.17849611e-01,  6.77801203e-03,
                9.59012657e-02,  3.82197052e-02,  1.00912027e-01,
               -5.27508818e-02, -8.89628287e-03,  3.10913268e-02,
                4.64734025e-02, -3.09740324e-02, -3.69712040e-02,
                2.27776356e-02, -7.77222356e-03,  9.15486142e-02,
               -5.27781248e-02,  1.34886764e-02,  5.86227626e-02,
               -2.72103888e-03,  3.79663706e-02,  2.04048716e-02,
               -3.46062705e-03,  2.95355171e-02, -7.40186051e-02,
               -4.59038615e-02, -4.81553227e-02,  6.62203506e-02,
                8.50068927e-02,  2.80059557e-02, -6.23318255e-02,
               -4.56236377e-02,  3.43918726e-02, -1.15749408e-02,
                1.17231403e-02,  2.63441587e-04, -1.97118506e-01,
               -9.20039695e-03, -1.81789584e-02,  1.50896031e-02,
               -4.31495048e-02,  7.06843436

In [36]:
lr_schedule=optax.warmup_cosine_decay_schedule(init_value=0, 
                                               peak_value=1, 
                                               warmup_steps=500, 
                                               decay_steps=8500, 
                                               end_value=1e-4)

In [41]:
import jaxlib, jax
jaxlib.__version__, jax.__version__, hk.__version__

('0.1.70', '0.2.19')

In [37]:
cnt = np.arange(0,9000,10)
lr = [lr_schedule(jnp.array([i])) for i in cnt]
plt.plot(cnt, lr)

NameError: name 'plt' is not defined

In [ ]:
{'loss': 2.4778389930725098, 'step': 0.0, 'steps_per_sec': 13.779059174634387}
{'loss': 1.3683964014053345, 'step': 500.0, 'steps_per_sec': 2.5767113856201855}
{'loss': 1.6631159782409668, 'step': 1000.0, 'steps_per_sec': 2.57725389686341}
{'loss': 1.3581798076629639, 'step': 1500.0, 'steps_per_sec': 2.5771785561488483}

In [ ]:
jax.core.gc.collect()

In [ ]:
jax.numpy.array([1,1])
#jax.device_get('gpu')
#jax.devices('gpu')
! nvcc --version

In [ ]:
from transformers import TrainingArguments
# TRAINING HYPERPARAMS
effective_batch_size = 32
max_steps = 4000
log_steps = 500
#n_step_examples = 8*500
batch_size = 16
grad_accumulation = effective_batch_size//batch_size
learning_rate = 5e-5
weight_decay = 0.01
warmup_ratio = 0.1
n_epochs = 3
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-{task}",
    evaluation_strategy = "steps",
    eval_steps = log_steps,
    logging_strategy = "steps",
    logging_steps = log_steps,
    save_strategy = "steps",
    save_steps = log_steps,
    learning_rate = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    #num_train_epochs = n_epochs,
    max_steps = max_steps,
    weight_decay = weight_decay,
    report_to = 'wandb', 
    gradient_accumulation_steps = grad_accumulation,
    warmup_steps = int(1.5*(log_steps)),
    # logging_steps = 100,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
)
#training_args

In [ ]:
data['test']=add_rle_word2(data['test'])
data

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=chunk_data["train"].remove_columns(['offset', 'text_id']), #.select(range(100)),
    eval_dataset=chunk_data["test"].remove_columns(['offset', 'text_id']), #.select(range(100)),
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=mk_binary_metric(chunk_ds=chunk_data['test'],
                                     orig_ds=data['test'])
    #,
    #                                 min_word_cnt=[0,10,5,10,3,10,6,6]), #.select(range(100))), 
)

In [ ]:
trainer.train()
trainer.save_model(model_path)

In [ ]:
# wandb.init()
import pickle
from feedback2021.postprocess import mk_prediction_transform
for k,v in chunk_data.items():
    predictions = trainer.predict(v.remove_columns(['offset', 'text_id']))
    prediction_transform = mk_binary_prediction_transform(chunk_ds=v, orig_ds=data[k])
    predictions = prediction_transform(predictions[0])
    with open(f'{k}_predictions.pkl','wb') as f:
        pickle.dump(predictions, f)
    wandb.save(f'{k}_predictions.pkl','./','now')

In [ ]:
# ! rm longformer-base-4096-token_classification/ -rf
#!rm distilroberta-base-token_classification -rf
! ls -sh distilroberta-base-1

In [ ]:
wandb.finish()

# From this point on there is no need to gpu!

In [ ]:
prediction_file = wandb.restore('predictions.pkl', run_path='prvi/huggingface/d17yam8m')

In [ ]:
import os

In [ ]:
prediction_file_name = 'test_predictions.pkl' #if os.path.exists('test_predictions.pkl') else prediction_file.name
with open(prediction_file_name, 'rb') as f:
    saved_predictions = pickle.load(f)

In [ ]:
chunk_ds, orig_ds = chunk_data['test'].to_dict(), data['test'].to_dict()

In [ ]:
def filter_prediction(prediction, min_word_cnt):
    return [b for b in prediction if b[2]-b[1]>=min_word_cnt[b[0]]]

In [ ]:
from tqdm.auto import tqdm
import pandas as pd
res = []
for mwc in tqdm(range(0,15)):
#     transform_prediction = mk_prediction_transform(chunk_ds=chunk_ds,
#                                                    orig_ds=orig_ds, min_word_cnt=mwc)
#     predictions = transform_prediction(saved_predictions[0])
    
    scores = np.array([metric.score_example(y_true=y_true, 
                                            y_pred=filter_prediction(y_pred,mwc)) 
              for y_pred,y_true in zip(saved_predictions,orig_ds['rle_word'])]).sum(axis=0)
    scores = [metric.f1(*score) for score in scores]
    f1 = sum(scores)/len(scores)
    scores = {id2label[i+1]:score for i,score in enumerate(scores)}
    scores['f1'] = f1
    scores['min_word_cnt']=mwc
    res.append(scores)

pd.DataFrame(res).set_index('min_word_cnt')

In [ ]:
mwc =[0, 10, 6, 15, 6, 11, 9, 8]

scores_ = np.array([metric.score_example(y_true=y_true, 
                                        y_pred=filter_prediction(y_pred,mwc)) 
              for y_pred,y_true in zip(saved_predictions,orig_ds['rle_word'])]).sum(axis=0)
scores = [metric.f1(*score) for score in scores_]
f1 = sum(scores)/len(scores)
scores = {id2label[i+1]:score for i,score in enumerate(scores)}
scores['f1'] = f1
    
# transform_prediction = mk_prediction_transform(chunk_ds=chunk_ds,
#                                                 orig_ds=orig_ds, 
#                                                min_word_cnt=[0, 10, 6, 15, 6, 11, 9, 8])
# predictions = transform_prediction(saved_predictions[0])
# scores = np.array([metric.score_example(y_true=y_true, y_pred=y_pred) 
#               for y_true,y_pred in zip(predictions,orig_ds['pred_range'])]).sum(axis=0)
# scores = [metric.f1(*score) for score in scores]
# f1 = sum(scores)/len(scores)
# scores = {id2label[i+1]:score for i,score in enumerate(scores)}
# scores['f1'] = f1
scores    

In [ ]:
scores_

In [ ]:
predictions = [ sorted(pred, key=lambda x: x[1]) for pred in saved_predictions]

In [ ]:
show_result(i=20, orig_ds=data['test'],predictions=predictions)

In [ ]:

saved_predictions[8],data['test'][8]['rle_token']

In [ ]:
scores = [metric.score_example(y_true=y_true, y_pred=y_pred) for y_true,y_pred in zip(predictions,orig_ds['pred_range'])]

In [ ]:
[metric.f1(*x) for x in np.array(scores).sum(axis=0)]

In [ ]:
idx = np.argsort(np.array(scores)[:,:,1:].sum(axis=(1,2)))[::-1]
idx[:10]